<img src="images/panel-logo.png" width="200px" align="right" vspace="20px"/>

# Intro to HoloViz and Panel

<hr>

HoloViz is a suite of high-level Python tools that are designed to work together to make visualizing data a breeze, from conducting exploratory data analysis to deploying complex dashboards.

The core HoloViz projects are as follows:

- [Panel](https://panel.holoviz.org): Create interactive dashboards in Jupyter notebooks or standalone apps
- [hvPlot](https://hvplot.holoviz.org): Quickly and interactively explore data with a familiar API
- [HoloViews](https://holoviews.org): Interactive plotting experience
- [GeoViews](http://geoviews.org): Geographic extension of HoloViews
- [Datashader](https://datashader.org): Render big data images in a browser
- [Lumen](https://lumen.holoviz.org/): Construct no-code dashboards from simple YAML specifications
- [Colorcet](https://colorcet.holoviz.org/): Plot with perceptually based colormaps
- [Param](https://param.holoviz.org): Declaratively code in Python

## What is Panel?

Today, the focus is on Panel.

Panel packs many pre-built frontend components that are **usable with Python**.

That means you can convert your static Python scripts into interactive ones--**no Javascript necessary**!

In [2]:
import panel as pn
pn.extension()

## Panel 101

Let's build an interactive app that allows the user to print a custom message.

To start, let's hard-code it to say `"Hello World"`.

In [3]:
print("Hello World!")

Hello World!


### Widget

We can give the user more control by introducing a `TextInput` widget.

In [4]:
message_input = pn.widgets.TextInput(value="Hello World!")

### Interactivity

Then, we can `pn.bind` the widget's `param.value` to the callback, `echo_message`, which simply echos the input value on change.

☝🏼 **Note:** It's important to prefix `value` with `param`-- without it, there will be no updates!

In [5]:
def echo_message(message):
    return f"<i>{message}</i>"

message_ref = pn.bind(echo_message, message=message_input.param.value)

### Layout

Next, create a simple layout to see the results.

Try typing something unique in the widget to see the message update!

In [6]:
pn.Column(message_input, message_ref)

Column
    [0] TextInput(value='Hello World!')
    [1] ParamFunction(function, _pane=Markdown, defer_load=False)

### Recap

To recap, we:

1. Instantiated a widget (`TextInput`).
2. Defined a function `echo_message`
3. Bound the function to the widget's *param* value
4. Laid out the widget and the bound reference

![recap](images/recap.png)

Here's all the code cells collected into one!

In [7]:
import panel as pn
pn.extension()

message_input = pn.widgets.TextInput(value="Hello World!")

def echo_message(message):
    return f"<i>{message}</i>"

message_ref = pn.bind(echo_message, message=message_input.param.value)

pn.Column(message_input, message_ref)

Column
    [0] TextInput(value='Hello World!')
    [1] ParamFunction(function, _pane=Markdown, defer_load=False)

### Your turn ✨ 

Doing this repeatedly is key to creating more complex apps with Panel, so let's do a quick exercise.

Your goal is to create a widget that will toggle the message to upper case if activated by filling out the ellipses (`...`)!

💡 **Hint:** Check out the [Component gallery](https://panel.holoviz.org/reference/index.html) to see what widgets are available to accomplish this goal (one of them starts with a `T`, but there are multiple solutions!).

In [8]:
import panel as pn
pn.extension()

message_input = pn.widgets.TextInput(value="Hello World!")
toggle_upper = pn.widgets.Toggle(name="Toggle Uppercase")

def echo_message(message, toggle_upper):
    if toggle_upper:
        message = message.upper()
    return f"<i>{message}</i>"

message_ref = pn.bind(echo_message, message=message_input.param.value, toggle_upper=toggle_upper.param.value)  # Fill this out

pn.Column(message_input, message_ref, toggle_upper)

Column
    [0] TextInput(value='Hello World!')
    [1] ParamFunction(function, _pane=Markdown, defer_load=False)
    [2] Toggle()

Congrats on building an interactive Panel app! 🎉

## Introducing Panel `ChatInterface`

Now, introducing `pn.chat.ChatInterface`, which is a component that packages all the steps you just learned to provide convenient features for developing a Chat UI with LLMs!

### Widget

Try typing a message and pressing enter to send!

In [9]:
chat = pn.chat.ChatInterface()
chat

ChatInterface(_button_data={'send': _ChatButtonData(i...}, _buttons={'send': Button(align='cen...}, _input_container=Row, _input_layout=Row, _placeholder=ChatMessage, _widgets={'ChatAreaInput': ChatArea...}, show_button_name=True, sizing_mode='stretch_width', widgets=[ChatAreaInput(css_classes...])

You might have noticed that it echoes the message you entered, but it doesn't reply... not fun (yet).

### Interactivity

To make it reply, all we have to do is set a `callback`, like `pn.bind`, but with a caveat: it needs these three arguments: `contents`, `user`, and `instance`.

Now when you try sending a message in the chat interface, it will be echoed back in italics!

In [10]:
def echo_message(contents: str, user: str, instance: pn.chat.ChatInterface):
    return f"<i>{contents}</i>"

# makes bot respond with your same message
chat.callback = echo_message
# makes bot respond all in caps and bold
chat.callback_user = "aggressive assistant"

### Streaming

You might have seen services, like OpenAI and Mistral, stream tokens as they arrive.

We can simulate streaming tokens by looping through the contents of the user's input, concatenating the characters to the final message, and `yield`ing it in italics.

Since there's no serious computation, it'll run too fast for us to perceive streaming--thus `time.sleep`.

Here's the latest code collected into one (and also `callback` within instantation).

In [11]:
import time
import panel as pn
pn.extension()

def stream_echo_message(contents: str, user: str, instance: pn.chat.ChatInterface):
    message = ""
    for char in contents:
        time.sleep(0.1)  # to simulate a serious computation - respond time per letter (imitation of a "real human" typing response?)
        message += char
        yield f"<i>{message}</i>"

chat = pn.chat.ChatInterface(callback=stream_echo_message)
chat

ChatInterface(_button_data={'send': _ChatButtonData(i...}, _buttons={'send': Button(align='cen...}, _input_container=Row, _input_layout=Row, _placeholder=ChatMessage, _widgets={'ChatAreaInput': ChatArea...}, callback=<function stream_echo_mess..., show_button_name=True, sizing_mode='stretch_width', widgets=[ChatAreaInput(css_classes...])

### More Interactivity

`pn.chat.ChatInterface` can be used with other widgets too!

Here, we include a `pn.widgets.FloatSlider` to control how long to wait between each character streamed.

In [12]:
import time
import panel as pn
pn.extension()

def stream_echo_message(contents: str, user: str, instance: pn.chat.ChatInterface):
    message = ""
    for char in contents:
        time.sleep(slider.value)  # to simulate a serious computation
        message += char
        yield f"<i>{message}</i>"

slider = pn.widgets.FloatSlider(start=0.01, value=0.5, name="Sleep (s)", align="center")
chat = pn.chat.ChatInterface(callback=stream_echo_message, min_height=350, styles={"background": "#03001c"})

pn.Column(slider, chat)

Column
    [0] FloatSlider(align='center', name='Sleep (s)', start=0.01, value=0.5)
    [1] ChatInterface(_button_data={'send': _ChatButtonData(i...}, _buttons={'send': Button(align='cen...}, _input_container=Row, _input_layout=Row, _placeholder=ChatMessage, _widgets={'ChatAreaInput': ChatArea...}, callback=<function stream_echo_mess..., min_height=350, show_button_name=True, sizing_mode='stretch_width', styles={'background': '#03001c'}, widgets=[ChatAreaInput(css_classes...])

### Other Inputs

`pn.chat.ChatInterface` also supports multi-modal inputs, like images, videos, PDFs, and more!

In [14]:
from io import BytesIO
import panel as pn

pn.extension()

def display_info(contents: BytesIO, user: str, instance: pn.chat.ChatInterface):
    size = len(contents.getvalue())
    return f"Size of input: {size / (1024 * 1024):.2f} MB"

file_input = pn.widgets.FileInput(accept=".jpeg,.png,.gif,.mp4,.pdf")
chat = pn.chat.ChatInterface(widgets=[file_input], callback=display_info, styles={"background": "#03001c"})
chat

ChatInterface(_button_data={'send': _ChatButtonData(i...}, _buttons={'send': Button(align='cen...}, _input_container=Row, _input_layout=Row, _placeholder=ChatMessage, _widgets={'FileInput': FileInput(ac...}, callback=<function display_info a..., show_button_name=True, sizing_mode='stretch_width', styles={'background': '#03001c'}, widgets=[FileInput(accept='.jpeg,....])

That's it for a crash course on Panel! These techniques, used repeatedly, will allow you to build increasingly complex web apps with just Python.

Check out the [`pn.chat.ChatInterface` documentation](https://panel.holoviz.org/reference/chat/ChatInterface.html), as well as [`pn.chat.ChatFeed`'s documentation](https://panel.holoviz.org/reference/chat/ChatFeed.html) that it inherits from.

You can also go through [more tutorials delving into Panel in general](https://panel.holoviz.org/tutorials/index.html), or check out the [app gallery](https://panel.holoviz.org/gallery/index.html).

There is also a [HoloViz Discourse](https://discourse.holoviz.org/) if you want to ask questions.

<hr>

**✨ Next: [Building a Panel Chat Applications with a Local LLM](05-panel-local-llm.ipynb) →**


💬 _Wish to continue discussions after the tutorial? Contact the presenters: [@pavithraes](https://github.com/pavithraes), [@dharhas](https://github.com/dharhas), [@ahuang11](https://github.com/ahuang11)_

<hr>